In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import jieba
import re

###模型建立與資料處裡
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score
from sklearn.preprocessing import LabelEncoder



In [3]:
#每次只要跑一次就好
#將dictionary改為個人工作路徑
#讀資料
raw_news = pd.read_csv('C:/dictionary/news.csv',encoding= 'big5')
#第一表格: 上市2018
stock_data = pd.read_excel('C:/dictionary/stock_data.xlsx',
                           sheet_name = '上市2018')
#第二表格: 上市2016
stock_data2 = pd.read_excel('C:/dictionary/stock_data.xlsx',
                           sheet_name = '上市2016')
#第三表格: 上市2017
stock_data3 = pd.read_excel('C:/dictionary/stock_data.xlsx',
                           sheet_name = '上市2017')
#合併df
stock_data = pd.concat([stock_data,stock_data2,stock_data3],axis = 0)
#重新排序
stock_data = stock_data.sort_values(['年月日','證券代碼'])

In [6]:
pd.value_counts(raw_news['post_time'].str.contains('2018'))

True     167754
False     79519
Name: post_time, dtype: int64

In [7]:
#查詢新聞篇數
pd.value_counts((raw_news['title'].str.contains('台積電')) | (raw_news['content'].str.contains('台積電') ))


False    229312
True      17961
dtype: int64

In [8]:
###變數輸入
stock_want_to_predict = '台積電'
N_days_after = '3'
percentage_threshold = 0.025 #預測漲跌門檻
signal_mode = 2 #漲跌模式選擇 #1 收盤價對收盤價 #2 隔天開盤對 N天後最高
tfidf_selection_mode = 3 #tfidf模式選擇 #1 tf前N名並取漲跌關鍵字差集 #2 tf-idf分數前N名並差集 #3 tf與卡方值均並列前N名並差集 
tfidf_N_words = 10000 #詞向量變數個數
vector_mode = 2  #詞向量模式選擇 #1 binary  #2 詞頻

In [9]:
#########欲選擇股價篩出
specific_stock_price = stock_data.loc[stock_data['證券代碼'].str.contains(stock_want_to_predict),:]

#賦予漲跌指標
#1天、2天、3天、4天、5天
if signal_mode == 1:
    specific_stock_price['1dayaf_close'] = specific_stock_price['收盤價(元)'].shift(-1)
    specific_stock_price['2dayaf_close'] = specific_stock_price['收盤價(元)'].shift(-2)
    specific_stock_price['3dayaf_close'] = specific_stock_price['收盤價(元)'].shift(-3)
    specific_stock_price['4dayaf_close'] = specific_stock_price['收盤價(元)'].shift(-4)
    specific_stock_price['5dayaf_close'] = specific_stock_price['收盤價(元)'].shift(-5)
    specific_stock_price = specific_stock_price.iloc[:-5,:]

    def get_sign(row,i):
        if ((row[f'{i}dayaf_close']) - row['收盤價(元)']) /row['收盤價(元)'] >percentage_threshold:
            return 'UP{}percent'.format(percentage_threshold*100)
        elif ((row[f'{i}dayaf_close']) - row['收盤價(元)']) /row['收盤價(元)']< -percentage_threshold:
            return 'DOWN{}percent'.format(percentage_threshold*100)
        else:
            return 'UNCHANGED'

        
    specific_stock_price['1dayaf_sign'] = specific_stock_price.apply(get_sign,i=1,axis = 1)
    specific_stock_price['2dayaf_sign'] = specific_stock_price.apply(get_sign,i=2,axis = 1)
    specific_stock_price['3dayaf_sign'] = specific_stock_price.apply(get_sign,i=3,axis = 1)
    specific_stock_price['4dayaf_sign'] = specific_stock_price.apply(get_sign,i=4,axis = 1)
    specific_stock_price['5dayaf_sign'] = specific_stock_price.apply(get_sign,i=5,axis = 1)    
        
        
elif signal_mode == 2:
    specific_stock_price['1dayaf_high'] = specific_stock_price['最高價(元)'].shift(-1)
    specific_stock_price['2dayaf_high'] = specific_stock_price['最高價(元)'].shift(-2)
    specific_stock_price['3dayaf_high'] = specific_stock_price['最高價(元)'].shift(-3)
    specific_stock_price['4dayaf_high'] = specific_stock_price['最高價(元)'].shift(-4)
    specific_stock_price['5dayaf_high'] = specific_stock_price['最高價(元)'].shift(-5)

    specific_stock_price['1dayaf_open'] = specific_stock_price['開盤價(元)'].shift(-1)
    
    specific_stock_price = specific_stock_price.iloc[:-5,:]
    
    def get_sign(row,i):
        if ((row[f'{i}dayaf_high']) - row['1dayaf_open']) /row['1dayaf_open'] >percentage_threshold:
            return 'UP{}percent'.format(percentage_threshold*100)
        elif ((row[f'{i}dayaf_high']) - row['1dayaf_open']) /row['1dayaf_open']< -percentage_threshold:
            return 'DOWN{}percent'.format(percentage_threshold*100)
        else:
            return 'UNCHANGED'
            return row.head()
    
    specific_stock_price['1dayaf_sign'] = specific_stock_price.apply(get_sign,i=1,axis = 1)
    specific_stock_price['2dayaf_sign'] = specific_stock_price.apply(get_sign,i=2,axis = 1)
    specific_stock_price['3dayaf_sign'] = specific_stock_price.apply(get_sign,i=3,axis = 1)
    specific_stock_price['4dayaf_sign'] = specific_stock_price.apply(get_sign,i=4,axis = 1)
    specific_stock_price['5dayaf_sign'] = specific_stock_price.apply(get_sign,i=5,axis = 1)
else:
    print('No this mode, following will go error')
    
    
#########尋找欲選股票新聞
news_stock = raw_news.loc[(raw_news['content'].str.contains(stock_want_to_predict)==True) | 
                          (raw_news['title'].str.contains(stock_want_to_predict)==True)  ,:]

#結合漲跌指標與新聞
#時間格式更改
news_stock['post_time'].dtype
news_stock['post_time'] = pd.to_datetime(news_stock['post_time'])
# 使用 dt.strftime() 格式化日期时间
news_stock['post_time'] = news_stock['post_time'].dt.strftime('%Y-%m-%d')

specific_stock_price['年月日'] = pd.to_datetime(specific_stock_price['年月日'])
# 使用 dt.strftime() 格式化日期时间
specific_stock_price['年月日']= specific_stock_price['年月日'].dt.strftime('%Y-%m-%d')


#針對不同天數進行新聞標註
#選想要的天數
news_for_stock_1 = pd.merge(news_stock,specific_stock_price.loc[:,['年月日','1dayaf_sign']],left_on= 'post_time', right_on= '年月日',
                     how = 'right') 
news_for_stock_1 = news_for_stock_1.rename(columns={'1dayaf_sign': 'Ndayaf_sign'})

news_for_stock_2 = pd.merge(news_stock,specific_stock_price.loc[:,['年月日','2dayaf_sign']],left_on= 'post_time', right_on= '年月日',
                     how = 'right') 
news_for_stock_2 = news_for_stock_2.rename(columns={'2dayaf_sign': 'Ndayaf_sign'})

news_for_stock_3 = pd.merge(news_stock,specific_stock_price.loc[:,['年月日','3dayaf_sign']],left_on= 'post_time', right_on= '年月日',
                     how = 'right') 
news_for_stock_3 = news_for_stock_3.rename(columns={'3dayaf_sign': 'Ndayaf_sign'})

news_for_stock_4 = pd.merge(news_stock,specific_stock_price.loc[:,['年月日','4dayaf_sign']],left_on= 'post_time', right_on= '年月日',
                     how = 'right') 
news_for_stock_4 = news_for_stock_4.rename(columns={'4dayaf_sign': 'Ndayaf_sign'})

news_for_stock_5 = pd.merge(news_stock,specific_stock_price.loc[:,['年月日','5dayaf_sign']],left_on= 'post_time', right_on= '年月日',
                     how = 'right') 
news_for_stock_5 = news_for_stock_5.rename(columns={'5dayaf_sign': 'Ndayaf_sign'})


C:\Users\shawn\AppData\Local\Temp\ipykernel_28352\1034110071.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific_stock_price['1dayaf_high'] = specific_stock_price['最高價(元)'].shift(-1)
C:\Users\shawn\AppData\Local\Temp\ipykernel_28352\1034110071.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  specific_stock_price['2dayaf_high'] = specific_stock_price['最高價(元)'].shift(-2)
C:\Users\shawn\AppData\Local\Temp\ipykernel_28352\1034110071.py:33: SettingWithCopyWarning: 
A value is trying to be set on a 

In [10]:
#變數命名
news_variable_name = "news_for_stock_{}".format(N_days_after)
news = locals()[news_variable_name]
news = news.loc[news['id'].isna()==False,:]
print('新聞標籤分布數量\n',pd.value_counts(news['Ndayaf_sign']))
print('股票漲跌日數分布\n',pd.value_counts(specific_stock_price['{}dayaf_sign'.format(N_days_after)]))


新聞標籤分布數量
 UNCHANGED         13571
UP2.5percent       2678
DOWN2.5percent      629
Name: Ndayaf_sign, dtype: int64
股票漲跌日數分布
 UNCHANGED         593
UP2.5percent      115
DOWN2.5percent     24
Name: 3dayaf_sign, dtype: int64


In [11]:
#將新聞區分成訓練資料與測試資料 - 全部進行訓練
train_news = news.sample(frac=0.8, random_state=42)
test_news = news.drop(train_news.index)
print('train_news:',len(train_news),'test_news:',len(test_news))
print('train_news有漲跌:',len(train_news[train_news['Ndayaf_sign']!='UNCHANGED']),'test_news有漲跌:',
      len(test_news[test_news['Ndayaf_sign']!='UNCHANGED']))


#將訓練新聞拆分為漲資料集與跌資料集
train_news_UP = train_news.loc[news['Ndayaf_sign'] == 'UP{}percent'.format(percentage_threshold*100),:]
train_news_DOWN = train_news.loc[news['Ndayaf_sign'] == 'DOWN{}percent'.format(percentage_threshold*100),:]
print('train_news_UP資料筆數:',len(train_news_UP),'train_news_DOWN資料筆數:',len(train_news_DOWN))

train_news: 13502 test_news: 3376
train_news有漲跌: 2628 test_news有漲跌: 679
train_news_UP資料筆數: 2140 train_news_DOWN資料筆數: 488


In [12]:
#要處裡空值行，不然preprocess_text會出錯
#將float型態轉為string
train_news_UP = train_news_UP.loc[train_news_UP['id'].isna()==False,:]
train_news_UP['content'] = train_news_UP['content'].astype(str)

# stop words
stop_words =set( [
    "的", "了", "和", "是", "在", "他", "她", "它", "我", "你",
    "我們", "你們", "他們", "她們", "它們", "這", "那", "這個", "那個", 
    "就是", "就", "吧", "呢", "啊", "哦", "哈", "哈哈", "嗯", "嗯嗯",
    "的話", "如果", "因為", "所以", "然後", "但是", "而且", "或者", "而是",
    "因此", "雖然", "但", "和", "與", "等", "及", "及其", "等等", "等等等",
    "這樣", "那樣", "這裡", "那裡", "這兒", "那兒", "這裡面", "那裡面",
    "這個時候", "那個時候", "現在", "以前", "以後", "一直", "一般", "通常",
    "總是", "有時候", "有的時候", "有時", "有的時", "有的", "一些", "一般來說",
    "總的來說", "總的說來", "總的來看", "因為這樣", "因為那樣", "例如", "譬如",
    "比如", "不過", "可是", "然而", "但是", "不過", "卻", "只是", "只不過",
    "不僅", "而且", "只有", "唯有", "不是", "不是因為", "不是因為而", "只有",
    "僅僅", "只不過", "只要", "只需", "只有", "只是", "隨著", "就是說", "對於",
    "對於這", "對於那", "對", "對了", "對的", "有", "沒有", "是不是", "是否",
    "就是說", "就是了", "就是的", "就是", "一般來說", "總的說來",
    # 常見的標點符號
    "。", "，", "！", "？", "：", "；", "（", "）", "&#8203;``【oaicite:0】``&#8203;", "《", "》", "「", "」", "『", "』"
    # ... 其他可能的停用詞
])

# 定義前處理公式
def preprocess_text(text):
    # 使用正則表達式刪除數字與英文
    text = re.sub(r'[0-9a-zA-Z]', '', text)
    # 使用 jieba 分詞並過濾掉停用詞
    words = [word for word in jieba.cut(text) if word not in stop_words]
    return ' '.join(words)



In [15]:

##先處裡過，變數一開始就能使用2、3 
# 對'content' 列進行前處理
train_news_UP = train_news_UP.loc[train_news_UP['id'].isna()==False,:]
train_news_UP['content'] = train_news_UP['content'].astype(str)
train_news_UP['processed_content'] = train_news_UP['content'].apply(preprocess_text)

# 获取处理后的文本数据
corpus_up = train_news_UP['processed_content'].tolist()

# 使用 TfidfVectorizer 進行 TF-IDF 分析
vectorizer_u = TfidfVectorizer(max_features = tfidf_N_words)  # 選擇前N個重要的特徵
tfidf_matrix_u = vectorizer_u.fit_transform(corpus_up)

# 獲取特徵名字列表
feature_names_u = vectorizer_u.get_feature_names_out()

# 創建 DataFrame
df_tfidf_u = pd.DataFrame(data=tfidf_matrix_u.toarray(), columns=feature_names_u)

# 計算每一列（詞彙）的 TF-IDF 分數之和，並根據這個總和降序排序
df_tfidf_sorted_u = df_tfidf_u.T.sum(axis=1).sort_values(ascending=False)

# 將排序後的詞彙索引取出
sorted_feature_indices_u = df_tfidf_sorted_u.index

# 根據排序後的詞彙索引重新排序 DataFrame
df_tfidf_sorted_u = df_tfidf_u[sorted_feature_indices_u]

#總和
# 對每一列計算總和
column_sum_u = df_tfidf_sorted_u.sum(axis=0)
df_tfidf_total_u = pd.DataFrame(column_sum_u, columns=['tf-idf'])

df_tfidf_total_u = df_tfidf_total_u.reset_index(drop = False)
# 顯示列總和
#print(df_tfidf_total.sort_values)
#df_tfidf_total.to_csv('df_tfidf_total.csv', index=True,encoding='utf-8')


#要處裡空值行，不然preprocess_text會出錯
train_news_DOWN = train_news_DOWN.loc[train_news_DOWN['id'].isna()==False,:]
train_news_DOWN['content'] = train_news_DOWN['content'].astype(str)

#改對下跌的文章進行處理

# 對'content' 列進行前處理
train_news_DOWN['processed_content'] = train_news_DOWN['content'].apply(preprocess_text)

# 获取处理后的文本数据
corpus_d = train_news_DOWN['processed_content'].tolist()


#使用 TfidfVectorizer 進行 TF-IDF 分析
vectorizer_d = TfidfVectorizer(max_features = tfidf_N_words)  # 選擇前N個重要的特徵
tfidf_matrix_d = vectorizer_d.fit_transform(corpus_d)





# 獲取特徵名字列表
feature_names_d = vectorizer_d.get_feature_names_out()

# 創建 DataFrame
df_tfidf_d = pd.DataFrame(data=tfidf_matrix_d.toarray(), columns=feature_names_d)

# 計算每一列（詞彙）的 TF-IDF 分數之和，並根據這個總和降序排序
df_tfidf_sorted_d = df_tfidf_d.T.sum(axis=1).sort_values(ascending=False)

# 將排序後的詞彙索引取出
sorted_feature_indices_d = df_tfidf_sorted_d.index

# 根據排序後的詞彙索引重新排序 DataFrame
df_tfidf_sorted_d = df_tfidf_d[sorted_feature_indices_d]

#總和
# 對每一列計算總和
column_sum_d = df_tfidf_sorted_d.sum(axis=0)
df_tfidf_total_d = pd.DataFrame(column_sum_d, columns=['tf-idf'])

df_tfidf_total_d = df_tfidf_total_d.reset_index(drop = False)
# 顯示列總和
#print(df_tfidf_total_d.sort_values)
df_tfidf_total_d.to_csv('df_tfidf_total_d.csv', index=True,encoding='utf-8')

#上漲下跌顯著詞彙
#篩選邏輯: tf-idf 各自前600名沒有出現在另外一個集合當中的詞彙
key_words_Down = list(set(df_tfidf_total_d['index']) - set(df_tfidf_total_u['index']))
key_words_Up = list(set(df_tfidf_total_u['index']) - set(df_tfidf_total_d['index']))
    
    
if tfidf_selection_mode == 1:
    ##選擇tf前N名並差集
    
    # 對'content' 列進行前處理
    train_news_UP = train_news_UP.loc[train_news_UP['id'].isna()==False,:]
    train_news_UP['content'] = train_news_UP['content'].astype(str)
    train_news_UP['processed_content'] = train_news_UP['content'].apply(preprocess_text)

    # 获取处理后的文本数据
    corpus_up = train_news_UP['processed_content'].tolist()

    # 使用 TfidfVectorizer 進行 TF-IDF 分析
    vectorizer_u = TfidfVectorizer(max_features = tfidf_N_words)  # 選擇前N個重要的特徵
    tfidf_matrix_u = vectorizer_u.fit_transform(corpus_up)

    # 獲取特徵名字列表
    feature_names_u = vectorizer_u.get_feature_names_out()

    # 創建 DataFrame
    df_tfidf_u = pd.DataFrame(data=tfidf_matrix_u.toarray(), columns=feature_names_u)

    # 計算每一列（詞彙）的 TF-IDF 分數之和，並根據這個總和降序排序
    df_tfidf_sorted_u = df_tfidf_u.T.sum(axis=1).sort_values(ascending=False)

    # 將排序後的詞彙索引取出
    sorted_feature_indices_u = df_tfidf_sorted_u.index

    # 根據排序後的詞彙索引重新排序 DataFrame
    df_tfidf_sorted_u = df_tfidf_u[sorted_feature_indices_u]

    #總和
    # 對每一列計算總和
    column_sum_u = df_tfidf_sorted_u.sum(axis=0)
    df_tfidf_total_u = pd.DataFrame(column_sum_u, columns=['tf-idf'])

    df_tfidf_total_u = df_tfidf_total_u.reset_index(drop = False)
    # 顯示列總和
    #print(df_tfidf_total.sort_values)
    #df_tfidf_total.to_csv('df_tfidf_total.csv', index=True,encoding='utf-8')


    #要處裡空值行，不然preprocess_text會出錯
    train_news_DOWN = train_news_DOWN.loc[train_news_DOWN['id'].isna()==False,:]
    train_news_DOWN['content'] = train_news_DOWN['content'].astype(str)

    #改對下跌的文章進行處理

    # 對'content' 列進行前處理
    train_news_DOWN['processed_content'] = train_news_DOWN['content'].apply(preprocess_text)

    # 获取处理后的文本数据
    corpus_d = train_news_DOWN['processed_content'].tolist()


    #使用 TfidfVectorizer 進行 TF-IDF 分析
    vectorizer_d = TfidfVectorizer(max_features = tfidf_N_words)  # 選擇前N個重要的特徵
    tfidf_matrix_d = vectorizer_d.fit_transform(corpus_d)





    # 獲取特徵名字列表
    feature_names_d = vectorizer_d.get_feature_names_out()

    # 創建 DataFrame
    df_tfidf_d = pd.DataFrame(data=tfidf_matrix_d.toarray(), columns=feature_names_d)

    # 計算每一列（詞彙）的 TF-IDF 分數之和，並根據這個總和降序排序
    df_tfidf_sorted_d = df_tfidf_d.T.sum(axis=1).sort_values(ascending=False)

    # 將排序後的詞彙索引取出
    sorted_feature_indices_d = df_tfidf_sorted_d.index

    # 根據排序後的詞彙索引重新排序 DataFrame
    df_tfidf_sorted_d = df_tfidf_d[sorted_feature_indices_d]

    #總和
    # 對每一列計算總和
    column_sum_d = df_tfidf_sorted_d.sum(axis=0)
    df_tfidf_total_d = pd.DataFrame(column_sum_d, columns=['tf-idf'])

    df_tfidf_total_d = df_tfidf_total_d.reset_index(drop = False)
    # 顯示列總和
    #print(df_tfidf_total_d.sort_values)
    df_tfidf_total_d.to_csv('df_tfidf_total_d.csv', index=True,encoding='utf-8')

    #上漲下跌顯著詞彙
    #篩選邏輯: tf-idf 各自前600名沒有出現在另外一個集合當中的詞彙
    key_words_Down = list(set(df_tfidf_total_d['index']) - set(df_tfidf_total_u['index']))
    key_words_Up = list(set(df_tfidf_total_u['index']) - set(df_tfidf_total_d['index']))
    print('up',len(key_words_Up),key_words_Up, 'down:',len(key_words_Down),key_words_Down)

elif tfidf_selection_mode == 2:
    ##使用tf-idf重要前N名並且差集
    #UP
    # 使用 TfidfVectorizer 計算 TF-IDF
    vectorizer_u = TfidfVectorizer(sublinear_tf = True)
    tfidf_matrix_u = vectorizer_u.fit_transform(corpus_up)

    # 獲取特徵詞彙表和對應的 TF-IDF 分數
    feature_names_u = vectorizer_u.get_feature_names_out()
    tfidf_scores_u = tfidf_matrix_u.max(axis=0).toarray()[0]


    # 將特徵和對應的 TF-IDF 分数组合成 DataFrame
    df_tfidf_u = pd.DataFrame({'feature': feature_names_u, 'tf-idf': tfidf_scores_u})

    # 按照 TF-IDF 分數進行排序
    df_tfidf_sorted_u = df_tfidf_u.sort_values(by='tf-idf', ascending=False)

    # 選擇前 N 個特徵
    selected_features_u = df_tfidf_sorted_u.head( tfidf_N_words )['feature'].tolist()


    #DOWN
    # 使用 TfidfVectorizer 計算 TF-IDF
    vectorizer_d = TfidfVectorizer(sublinear_tf = True)
    tfidf_matrix_d = vectorizer_d.fit_transform(corpus_d)

    # 獲取特徵詞彙表和對應的 TF-IDF 分數
    feature_names_d = vectorizer_d.get_feature_names_out()
    tfidf_scores_d = tfidf_matrix_d.max(axis=0).toarray()[0]

    # 將特徵和對應的 TF-IDF 分数组合成 DataFrame
    df_tfidf_d = pd.DataFrame({'feature': feature_names_d, 'tf-idf': tfidf_scores_d})

    # 按照 TF-IDF 分數進行排序
    df_tfidf_sorted_d = df_tfidf_d.sort_values(by='tf-idf', ascending=False)

    # 選擇前 1500 個特徵
    selected_features_d = df_tfidf_sorted_d.head( tfidf_N_words )['feature'].tolist()


    #篩出詞彙差集
    key_words_Down = list(set(selected_features_d) - set(selected_features_u))
    key_words_Up = list(set(selected_features_u) - set(selected_features_d))
    print('up',len(key_words_Up),key_words_Up, 'down:',len(key_words_Down),key_words_Down)

else:
    ###加入卡方值來選取詞彙
    ##DOWN
    # 使用 TfidfVectorizer 進行 TF-IDF 分析
    vectorizer_d = TfidfVectorizer(sublinear_tf = True)  
    tfidf_matrix_d = vectorizer_d.fit_transform(corpus_d)
    # 獲取特徵名字列表
    feature_names_d = vectorizer_d.get_feature_names_out()

    # 創建 DataFrame
    df_tfidf_d = pd.DataFrame(data=tfidf_matrix_d.toarray(), columns=feature_names_d)

    # 計算每一列（詞彙）的 TF-IDF 分數之和，並根據這個總和降序排序
    df_tfidf_sorted_d = df_tfidf_d.T.sum(axis=1).sort_values(ascending=False)

    # 將排序後的詞彙索引取出
    sorted_feature_indices_d = df_tfidf_sorted_d.index

    # 根據排序後的詞彙索引重新排序 DataFrame
    df_tfidf_sorted_d = df_tfidf_d[sorted_feature_indices_d]
    #總和
    # 對每一列計算總和
    column_sum_d = df_tfidf_sorted_d.sum(axis=0)
    df_tfidf_total_d = pd.DataFrame(column_sum_d, columns=['tf-idf'])

    df_tfidf_total_d = df_tfidf_total_d.reset_index(drop = False)

    #類別中down df 
    df_tfidf_total_d['類別df'] = (df_tfidf_sorted_d.T>0).sum(axis=1).to_list()
    key_words_Down = df_tfidf_total_d['index']


    ##UP
    # 使用 TfidfVectorizer 進行 TF-IDF 分析
    vectorizer_u = TfidfVectorizer(sublinear_tf = True)  
    tfidf_matrix_u = vectorizer_u.fit_transform(corpus_up)

    # 獲取特徵名字列表
    feature_names_u = vectorizer_u.get_feature_names_out()

    # 創建 DataFrame
    df_tfidf_u = pd.DataFrame(data=tfidf_matrix_u.toarray(), columns=feature_names_u)

    # 計算每一列（詞彙）的 TF-IDF 分數之和，並根據這個總和降序排序
    df_tfidf_sorted_u = df_tfidf_u.T.sum(axis=1).sort_values(ascending=False)

    # 將排序後的詞彙索引取出
    sorted_feature_indices_u = df_tfidf_sorted_u.index

    # 根據排序後的詞彙索引重新排序 DataFrame
    df_tfidf_sorted_u = df_tfidf_u[sorted_feature_indices_u]

    #總和
    # 對每一列計算總和
    column_sum_u = df_tfidf_sorted_u.sum(axis=0)
    df_tfidf_total_u = pd.DataFrame(column_sum_u, columns=['tf-idf'])

    df_tfidf_total_u = df_tfidf_total_u.reset_index(drop = False)

    #類別中up df 
    df_tfidf_total_u['類別df'] = (df_tfidf_sorted_u.T>0).sum(axis=1).to_list()
    key_words_Up = df_tfidf_total_u['index']

    ###全df
    df_tfidf_total_d['index']
    corpus = corpus_up + corpus_d
    keywords = set(key_words_Up.to_list()+key_words_Down.to_list())

    # 使用 TfidfVectorizer 進行 TF-IDF 分析
    vectorizer = TfidfVectorizer(vocabulary= keywords)  
    tfidf_matrix = vectorizer.fit_transform(corpus)

    # 獲取特徵名字列表
    feature_names = vectorizer.get_feature_names_out()

    # 創建 DataFrame
    df_tfidf = pd.DataFrame(data=tfidf_matrix.toarray(), columns=feature_names)

    # 計算每一列（詞彙）的 TF-IDF 分數之和，並根據這個總和降序排序
    df_tfidf_sorted = df_tfidf.T.sum(axis=1).sort_values(ascending=False)

    # 將排序後的詞彙索引取出
    sorted_feature_indices = df_tfidf_sorted.index

    # 根據排序後的詞彙索引重新排序 DataFrame
    df_tfidf_sorted = df_tfidf[sorted_feature_indices]

    #計算總df
    total_df = pd.DataFrame((df_tfidf_sorted.T>0).sum(axis=1),columns = ['全部df']).reset_index()

    #for down
    df_tfidf_total_d = pd.merge(df_tfidf_total_d,total_df.loc[:,['index','全部df']], on ='index', how = 'left')
    Expected_df_d =  (df_tfidf_total_d['全部df']/len(corpus)) * len(corpus_d)
    Observed_df_d = df_tfidf_total_d['類別df'] 
    df_tfidf_total_d['chi-square'] = ((( Observed_df_d - Expected_df_d )**2)/Expected_df_d)
    print(df_tfidf_total_d)

    #for up
    df_tfidf_total_u = pd.merge(df_tfidf_total_u,total_df.loc[:,['index','全部df']], on ='index', how = 'left')
    Expected_df_u =  (df_tfidf_total_u['全部df']/len(corpus)) * len(corpus_up)
    Observed_df_u = df_tfidf_total_u['類別df'] 
    df_tfidf_total_u['chi-square'] = ((( Observed_df_u - Expected_df_u )**2)/Expected_df_u)
    print(df_tfidf_total_u)


    ###挑選詞彙
    #chi-square 前 N 名且 tf-idf也前 N 名的詞彙

    #down
    top_chi_square_d = df_tfidf_total_d.nlargest(tfidf_N_words,'chi-square')
    top_tfidf_d = df_tfidf_total_d.nlargest(tfidf_N_words, 'tf-idf')
    top_combined_d = pd.merge(top_chi_square_d, top_tfidf_d, on='index')
    print( '跌關鍵字:' , top_combined_d['index'])
    key_words_Down = top_combined_d['index']
    #up
    top_chi_square_u = df_tfidf_total_u.nlargest(tfidf_N_words,'chi-square')
    top_tfidf_u = df_tfidf_total_u.nlargest(tfidf_N_words, 'tf-idf')
    top_combined_u = pd.merge(top_chi_square_u, top_tfidf_u, on='index')
    key_words_Up = top_combined_u['index']
    print( '漲關鍵字:' , top_combined_u['index'])

    key_words_Down = list(set(top_combined_d['index']) - set(top_combined_u['index']))
    key_words_Up = list(set(top_combined_u['index']) - set(top_combined_d['index']))


#pd.DataFrame(key_words_Down).to_csv('key_words_Down.csv', index=True,encoding='utf-8')
#pd.DataFrame(key_words_Up).to_csv('key_words_Up.csv', index=True,encoding='utf-8')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\shawn\AppData\Local\Temp\jieba.cache
Loading model cost 0.410 seconds.
Prefix dict has been built successfully.


      index     tf-idf  類別df  全部df  chi-square
0       台積電  14.444456   332  1793    0.002692
1        台股  13.309319   267  1542    1.305990
2        指數  12.902290   242  1446    2.617599
3        億元  12.288427   252  1428    0.654003
4        外資   9.757621   193  1106    0.745775
...     ...        ...   ...   ...         ...
14175    稱其   0.058460     1     2    1.064008
14176    量化   0.058460     1     4    0.089082
14177    現好   0.058460     1     1    3.570938
14178    工資   0.058460     1     2    1.064008
14179    蓄率   0.058460     1     1    3.570938

[14180 rows x 5 columns]
      index     tf-idf  類別df  全部df  chi-square
0       台積電  58.612336  1461  1793    0.000614
1        台股  58.081008  1275  1542    0.297814
2        指數  56.649777  1204  1446    0.596910
3        億元  51.947262  1176  1428    0.149137
4        市場  41.284974  1063  1278    0.478495
...     ...        ...   ...   ...         ...
37611   移民法   0.040690     1     1    0.042345
37612   強貿易   0.040690     1     1

In [16]:
#查詢特定詞彙卡方值
print('特定詞彙卡方值:',df_tfidf_total_d.loc[df_tfidf_total_d['index']== '投資人','chi-square'])

#查看 chisquare 分布
pd.set_option('display.float_format', '{:.4f}'.format)
print('\nchisquare 分布:\n',pd.value_counts(df_tfidf_total_d.sort_values('chi-square',ascending= False)['chi-square']>1))

#查看 keywords長度
print('\nkey_words_Down長度:',len(key_words_Down))
print('key_words_Up長度:',len(key_words_Up))



特定詞彙卡方值: 271    2.459466
Name: chi-square, dtype: float64

chisquare 分布:
 True     8695
False    5485
Name: chi-square, dtype: int64

key_words_Down長度: 3962
key_words_Up長度: 3782


In [17]:
if vector_mode == 1:

    # 使用 TfidfVectorizer
    # TfidfVectorizer裡面 use_idf and norm 要記得設定，不然只是tf binary 不是整個output
    #使用訓練新聞集建立訓練矩陣

    #對'content' 列進行前處理
    train_news['processed_content'] = train_news['content'].astype(str).apply(preprocess_text)

    corpus = train_news['processed_content'].tolist()

    keywords = list(set(key_words_Up+key_words_Down))
    vectorizer = TfidfVectorizer(vocabulary=keywords, binary=True,use_idf = False,norm = None) 


    tfidf_matrix = vectorizer.fit_transform(corpus)

    # 將 tfidf_matrix 轉換為 DataFrame
    df_wv_space_train = pd.DataFrame(data=tfidf_matrix.toarray(), columns=keywords)
    df_wv_space_train = df_wv_space_train.astype(int)
    # 顯示結果
    print(df_wv_space_train)


    #顯示漲跌新聞比例
    print('訓練新聞-漲跌新聞比例\n',pd.value_counts(df_wv_space_train.iloc[:,0]))

    #加上漲跌標籤
    #3天後股價上漲下跌超過2.5%
    df_wv_space_train['Ndayaf_sign'] = train_news['Ndayaf_sign'].to_list()





    #使用測試新聞集建立測試矩陣
    #對'content' 列進行前處理
    test_news['processed_content'] = test_news['content'].astype(str).apply(preprocess_text)

    corpus = test_news['processed_content'].tolist()
    keywords = list(set(key_words_Up+key_words_Down))
    vectorizer = TfidfVectorizer(vocabulary=keywords, binary=True,use_idf = False,norm = None) 


    tfidf_matrix = vectorizer.fit_transform(corpus)

    # 將 tfidf_matrix 轉換為 DataFrame
    df_wv_space_test = pd.DataFrame(data=tfidf_matrix.toarray(), columns=keywords)
    df_wv_space_test = df_wv_space_test.astype(int)
    # 顯示結果
    print(df_wv_space_test)


    #顯示漲跌新聞比例
    print('測試新聞-漲跌新聞比例\n',pd.value_counts(df_wv_space_test.iloc[:,0]))

    #加上漲跌標籤
    #3天後股價上漲下跌超過2.5%

    df_wv_space_test['Ndayaf_sign'] = test_news['Ndayaf_sign'].to_list()


    #匯出csv
    #df_wv_space_test.to_csv('df_wv_space_test.csv', index=True,encoding='utf-8')
    
elif vector_mode == 2:
    # 使用 TfidfVectorizer
    # TfidfVectorizer裡面 use_idf and norm 要記得設定，不然只是tf binary 不是整個output
    #使用訓練新聞集建立訓練矩陣

    #對'content' 列進行前處理
    train_news['processed_content'] = train_news['content'].astype(str).apply(preprocess_text)

    corpus = train_news['processed_content'].tolist()

    keywords = list(set(key_words_Up+key_words_Down))
    count_vectorizer = CountVectorizer(vocabulary=keywords)

    tfidf_matrix = count_vectorizer.fit_transform(corpus)

    # 將 tfidf_matrix 轉換為 DataFrame
    df_wv_space_train = pd.DataFrame(data=tfidf_matrix.toarray(), columns=keywords)
    df_wv_space_train = df_wv_space_train.astype(int)
    # 顯示結果
    print(df_wv_space_train)


    #顯示漲跌新聞比例
    print('訓練新聞-漲跌新聞比例\n',pd.value_counts(df_wv_space_train.iloc[:,0]))

    #加上漲跌標籤
    #3天後股價上漲下跌超過2.5%
    df_wv_space_train['Ndayaf_sign'] = train_news['Ndayaf_sign'].to_list()





    #使用測試新聞集建立測試矩陣
    #對'content' 列進行前處理
    test_news['processed_content'] = test_news['content'].astype(str).apply(preprocess_text)

    corpus = test_news['processed_content'].tolist()
    keywords = list(set(key_words_Up+key_words_Down))
    count_vectorizer = CountVectorizer(vocabulary=keywords)


    tfidf_matrix = count_vectorizer.fit_transform(corpus)

    # 將 tfidf_matrix 轉換為 DataFrame
    df_wv_space_test = pd.DataFrame(data=tfidf_matrix.toarray(), columns=keywords)
    df_wv_space_test = df_wv_space_test.astype(int)
    # 顯示結果
    print(df_wv_space_test)


    #顯示漲跌新聞比例
    print('測試新聞-漲跌新聞比例\n',pd.value_counts(df_wv_space_test.iloc[:,0]))

    #加上漲跌標籤
    #3天後股價上漲下跌超過2.5%

    df_wv_space_test['Ndayaf_sign'] = test_news['Ndayaf_sign'].to_list()


    #匯出csv
    #df_wv_space_test.to_csv('df_wv_space_test.csv', index=True,encoding='utf-8')
    
else:
    print('no this mode')


       軍備  日加權  將與會  靠奈米  突發  急升  美中爭  營收表現  風國際  為點  ...  關心  帶動華新  還重  台燿大漲  \
0       0    0    0    0   0   0    0     0    0   0  ...   0     0   0     0   
1       0    0    0    0   0   0    0     0    0   0  ...   0     0   0     0   
2       0    0    0    0   0   0    0     0    0   0  ...   0     0   0     0   
3       0    0    0    0   0   0    0     0    0   0  ...   0     0   0     0   
4       0    0    0    0   0   0    0     0    0   0  ...   0     0   0     0   
...    ..  ...  ...  ...  ..  ..  ...   ...  ...  ..  ...  ..   ...  ..   ...   
13497   0    0    0    0   0   0    0     0    0   0  ...   0     0   0     0   
13498   0    0    0    0   0   0    0     0    0   0  ...   0     0   0     0   
13499   0    0    0    0   0   0    0     0    0   0  ...   0     0   0     0   
13500   0    0    0    0   0   0    0     0    0   0  ...   0     0   0     0   
13501   0    0    0    0   0   0    0     0    0   0  ...   0     0   0     0   

       投信將  外傳  地震  華碩漲  大勝

In [18]:
max(df_wv_space_test.iloc[:,:])

'龐大賣'

In [19]:
##檢查用
#檢查corpus中特定詞彙出現的index
test_word_in_corpus = df_wv_space_test.columns[3]
contains_search = [test_word_in_corpus in text for text in corpus]

# 输出结果
#print(contains_search)
true_indices = [index for index, value in enumerate(contains_search) if value]

# 输出结果
print(true_indices)

#檢查wv_space有沒有對應到上面的index
df_wv_space_test.loc[[19,386, 65, 84, 124, 128, 20],test_word_in_corpus]

[]


19     0
386    0
65     0
84     0
124    0
128    0
20     0
Name: 靠奈米, dtype: int32

In [21]:
###模型建立與資料處裡
###拆分資料
y_train = df_wv_space_train['Ndayaf_sign']
y_test = df_wv_space_test['Ndayaf_sign']
X_train = df_wv_space_train.drop('Ndayaf_sign',axis = 1)
X_test = df_wv_space_test.drop('Ndayaf_sign',axis = 1)




In [22]:
##利用股價資料與衍生指標計算漲跌機率值，再將預測機率作為變數放入後續模型
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

# 假設數據框名稱為 data
# 選取需要的特徵
features = ['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交量(千股)', '成交值(千元)', '成交筆數(筆)', '流通在外股數(千股)']

# 選取目標變數
target = '3dayaf_sign'

# 创建 XGBoost 分类器
# 創建特徵矩陣 X 和目標變數 y
X = specific_stock_price[features]
y = specific_stock_price[target]

# 新增特徵
X['volume_diff'] = specific_stock_price['成交量(千股)'] - specific_stock_price['成交量(千股)'].rolling(5).mean()
X['average_price_5d'] = specific_stock_price['收盤價(元)'].rolling(5).mean()
X['close_price_pct_change'] = (specific_stock_price['收盤價(元)'] - specific_stock_price['收盤價(元)'].rolling(5).mean()) / specific_stock_price['收盤價(元)'].rolling(5).mean()
X['volume_pct_change'] = (specific_stock_price['成交量(千股)'] - specific_stock_price['成交量(千股)'].rolling(5).mean()) / specific_stock_price['成交量(千股)'].rolling(5).mean()
X['close_price_pct_change_prev_day'] = specific_stock_price['收盤價(元)'].pct_change()


# Additional derived indicators
X['short_ma_close'] = specific_stock_price['收盤價(元)'].rolling(window=10).mean()
X['medium_ma_close'] = specific_stock_price['收盤價(元)'].rolling(window=20).mean()
X['long_ma_close'] = specific_stock_price['收盤價(元)'].rolling(window=40).mean()




# 去除包含空值的行
X = X.reset_index(drop=True).dropna()

y = y.reset_index(drop=True)[X.index]  # 确保目标变量与特征对应


##利用logistic regression 賦予每天漲超過 特定幅度 的機率
##訓練測試區
# instantiate the model (using the default parameters)

y_logistic = [0 if y == 'UNCHANGED' else 1 if y == 'UP{}percent'.format(percentage_threshold * 100) else 0 for y in y]
# 將數據分為訓練集和測試集
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X, y_logistic, test_size=0.1, random_state=42)

logreg = LogisticRegression(solver='lbfgs', max_iter=1000)



# 正規化特徵矩陣 X
scaler = StandardScaler()
Xtrain1_normalized = scaler.fit_transform(X_train_1)
Xtrain1_normalized = pd.DataFrame(Xtrain1_normalized, columns=X_train_1.columns)


# fit the model with data
logreg.fit(Xtrain1_normalized, y_train_1)

# 正規化特徵矩陣 X
scaler = StandardScaler()
Xtest1_normalized = scaler.fit_transform(X_test_1)
Xtest1_normalized = pd.DataFrame(Xtest1_normalized, columns=X_test_1.columns)


y_pred_logistic = logreg.predict_proba(Xtest1_normalized)
y_pred_logistic = y_pred_logistic[:, 1]

# Assuming y_probabilities_class_1 contains the predicted probabilities for class 1

# Convert probabilities to binary predictions
y_pred_binary = np.where(y_pred_logistic > 0.23, 1, 0)

# Assuming y_test contains the true labels for your test set

# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test_1, y_pred_binary)

print("Confusion Matrix:")
print(conf_matrix)



##全訓練 重新餵入詞向量model
#找出如何對上新聞的日期
# instantiate the model (using the default parameters)

y_logistic = [0 if y == 'UNCHANGED' else 1 if y == 'UP{}percent'.format(percentage_threshold * 100) else 0 for y in y]
# 將數據分為訓練集和測試集

logreg = LogisticRegression(solver='lbfgs', max_iter=1000)

# fit the model with data
logreg.fit(X, y_logistic)

#獲得預測機率
y_pred_logistic = logreg.predict_proba(X)
y_pred_logistic = y_pred_logistic[:, 1]


#將日期與預測機率值合併
y_pred_logistic_prob = pd.DataFrame({'年月日':specific_stock_price.reset_index(drop = True).loc[X.index,'年月日'], 'Prob': y_pred_logistic})


#將合併後的日期與預測機率值 再分別與訓練與測試新聞集合併
logistic_prob_for_train_news =  pd.merge(train_news,y_pred_logistic_prob,left_on= 'post_time',right_on = '年月日', how = 'left')[['post_time','Prob','Ndayaf_sign']]
logistic_prob_for_train_news['Prob'] = (logistic_prob_for_train_news['Prob']*100)**2
logistic_prob_for_train_news.head()
logistic_prob_for_test_news =  pd.merge(test_news,y_pred_logistic_prob,left_on= 'post_time',right_on = '年月日', how = 'left')[['post_time','Prob','Ndayaf_sign']]
logistic_prob_for_test_news['Prob'] = (logistic_prob_for_test_news['Prob']*100)**2
logistic_prob_for_test_news.head()

print('done',logistic_prob_for_test_news)

Confusion Matrix:
[[55  2]
 [10  3]]
done        post_time     Prob     Ndayaf_sign
0     2016-01-04      NaN  DOWN2.5percent
1     2016-01-04      NaN  DOWN2.5percent
2     2016-01-04      NaN  DOWN2.5percent
3     2016-01-04      NaN  DOWN2.5percent
4     2016-01-05      NaN       UNCHANGED
...          ...      ...             ...
3371  2018-12-22 250.3721       UNCHANGED
3372  2018-12-22 250.3721       UNCHANGED
3373  2018-12-22 250.3721       UNCHANGED
3374  2018-12-22 250.3721       UNCHANGED
3375  2018-12-22 250.3721       UNCHANGED

[3376 rows x 3 columns]


C:\Users\shawn\AppData\Local\Temp\ipykernel_28352\488235730.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['volume_diff'] = specific_stock_price['成交量(千股)'] - specific_stock_price['成交量(千股)'].rolling(5).mean()
C:\Users\shawn\AppData\Local\Temp\ipykernel_28352\488235730.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['average_price_5d'] = specific_stock_price['收盤價(元)'].rolling(5).mean()
C:\Users\shawn\AppData\Local\Temp\ipykernel_28352\488235730.py:27: SettingWithCopyWarning: 
A value is trying 

In [23]:
###Model 3 with logistic prob.
class_weights = {'UP{}percent'.format(percentage_threshold*100): 0.7, 'UP{}percent'.format(percentage_threshold*100): 0.2,
                'UNCHANGED':0.1}


y_pred_logistic
# Instantiate the random forest classifier with key parameters
clf_rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    #class_weight = class_weights
)


#train with prob
X_train_with_prob = X_train
X_train_with_prob['Prob'] = logistic_prob_for_train_news['Prob']
y_train_with_prob = y_train.loc[X_train_with_prob['Prob'].isna()==False]
X_train_with_prob = X_train_with_prob.loc[X_train_with_prob['Prob'].isna()==False,:]

clf_rf.fit(X_train_with_prob, y_train_with_prob)

#train only by word vector
#clf_rf.fit(X_train, y_train)


# 预测
#test with prob
X_test_with_prob = X_test
X_test_with_prob['Prob'] = logistic_prob_for_test_news['Prob']
y_test_with_prob = y_test.loc[X_test_with_prob['Prob'].isna()==False]
X_test_with_prob = X_test_with_prob.loc[X_test_with_prob['Prob'].isna()==False,:]


y_pred3 = clf_rf.predict(X_test_with_prob)



# Evaluate the performance on the test set to compare

print('accuracy_score: {:.3f}'.format(accuracy_score(y_test_with_prob, y_pred3)))
print('recall_score: {:.3f}'.format(recall_score(y_test_with_prob, y_pred3, average='micro')))
print('precision_score: {:.3f}'.format(precision_score(y_test_with_prob, y_pred3, average='micro')))


#三維
# 計算混淆矩陣
conf_matrix = confusion_matrix(y_test_with_prob, y_pred3, labels=['UP{}percent'.format(percentage_threshold*100), 'DOWN{}percent'.format(percentage_threshold*100), 'UNCHANGED'])

index_names = ['Actual UP2percent', 'Actual DOWN2percent', 'Actual UNCHANGED']
column_names = ['Predicted UP2percent', 'Predicted DOWN2percent', 'Predicted UNCHANGED']

# 將混淆矩陣包裝在 DataFrame 中
confusion_df = pd.DataFrame(conf_matrix, index=index_names, columns=column_names)

# 打印混淆矩陣
print("Confusion Matrix:")
print(confusion_df)

 

accuracy_score: 0.873
recall_score: 0.873
precision_score: 0.873
Confusion Matrix:
                     Predicted UP2percent  Predicted DOWN2percent  \
Actual UP2percent                     238                       0   
Actual DOWN2percent                     1                      61   
Actual UNCHANGED                       55                      10   

                     Predicted UNCHANGED  
Actual UP2percent                    275  
Actual DOWN2percent                   75  
Actual UNCHANGED                    2567  


In [24]:
#將預測轉為日期維度
y_pred = y_pred3

date_prediction = pd.DataFrame(logistic_prob_for_test_news[logistic_prob_for_test_news['Prob'].isna()==False]['post_time'].to_list(),y_pred).reset_index(drop=False)
date_prediction.loc[date_prediction.loc[:,'index'].str.contains('UP'),'predict_sign_sum'] = 1
date_prediction.loc[date_prediction.loc[:,'index'].str.contains('UNCHANGED'),'predict_sign_sum'] = 0
date_prediction.loc[date_prediction.loc[:,'index'].str.contains('DOWN'),'predict_sign_sum'] = -1
date_prediction.rename(columns={'index': 'Prediction'}, inplace=True)
date_prediction.rename(columns={0: 'Date'}, inplace=True)
date_prediction.head()

# 將整天的新聞預測合併，觀察是預測漲的新聞多、跌的新聞多，還是不變的新聞多
date_prediction_gb = date_prediction.groupby(by = 'Date')['predict_sign_sum'].sum()
label =  specific_stock_price.loc[specific_stock_price['年月日']>=min(test_news['年月日']) ,['年月日','{}dayaf_sign'.format(N_days_after)]] #真正股價指標
date_prediction_gb = pd.merge(date_prediction_gb,label, left_on = 'Date', right_on = '年月日' ,how = 'right')
date_prediction_gb = date_prediction_gb.iloc[:,[1,0,2]]

date_prediction_gb['predict_sign_sum'] = date_prediction_gb['predict_sign_sum'].fillna(0)
date_prediction_gb.loc[date_prediction_gb['predict_sign_sum']>=2,'final_predict_sign'] = 'UP{}percent'.format(percentage_threshold*100)
date_prediction_gb.loc[(date_prediction_gb['predict_sign_sum']>=0)&(date_prediction_gb['predict_sign_sum']<=1),'final_predict_sign'] = 'UNCHANGED'
date_prediction_gb.loc[date_prediction_gb['predict_sign_sum']<=-1,'final_predict_sign'] = 'DOWN{}percent'.format(percentage_threshold*100)
date_prediction_gb.head()


##計算各觀測指標
print('起始日期:',min(date_prediction_gb['年月日']),'終止日期:',max(date_prediction_gb['年月日']),
      '有效日數:',len(date_prediction_gb))
print(pd.value_counts(date_prediction_gb['final_predict_sign']))
effective_sign_1 = pd.value_counts(date_prediction_gb['final_predict_sign']).get(1, 0)  # UP
effective_sign_2 = pd.value_counts(date_prediction_gb['final_predict_sign']).get(2, 0)  # DOWN
noneffective_sign = pd.value_counts(date_prediction_gb['final_predict_sign']).get(0, 0)  # UNCHANGED

effective_ratio = (effective_sign_1 ) / (effective_sign_1 + effective_sign_2 + noneffective_sign)
print('\n出手率(漲) effective_ratio:\n','{:.2f}%'.format(effective_ratio*100))


# 計算出手混淆矩陣
conf_matrix = confusion_matrix(date_prediction_gb['{}dayaf_sign'.format(N_days_after)],date_prediction_gb['final_predict_sign'], 
                               labels=['UP{}percent'.format(percentage_threshold*100), 
                                       'DOWN{}percent'.format(percentage_threshold*100), 'UNCHANGED'])

index_names = ['Actual UP', 'Actual DOWN', 'Actual UNCHANGED']
column_names = ['Predicted UP', 'Predicted DOWN', 'Predicted UNCHANGED']

# 將混淆矩陣包裝在 DataFrame 中
confusion_df = pd.DataFrame(conf_matrix, index=index_names, columns=column_names)

# 打印混淆矩陣
print("Confusion Matrix:")
print(confusion_df)

#如有出手的指標
Accuracy = (confusion_df.iloc[0,0] + confusion_df.iloc[1,1]) / confusion_df.iloc[:,[0,1]].sum().sum()
print('')
print('如有出手的指標')
print('Accuracy:',Accuracy)

Precision = confusion_df.iloc[0,0]  / (confusion_df.iloc[0,0] + confusion_df.iloc[1,0]+ confusion_df.iloc[2,0])
print('')
print('Precision:',Precision)

Recall = confusion_df.iloc[0,0]  / (confusion_df.iloc[0,0] + confusion_df.iloc[0,1] +  confusion_df.iloc[0,2])
print('')
print('Recall:',Recall)

起始日期: 2016-01-04 終止日期: 2018-12-22 有效日數: 732
UNCHANGED         666
UP2.5percent       44
DOWN2.5percent     22
Name: final_predict_sign, dtype: int64

出手率(漲) effective_ratio:
 6.01%
Confusion Matrix:
                  Predicted UP  Predicted DOWN  Predicted UNCHANGED
Actual UP                   37               0                   78
Actual DOWN                  0              12                   12
Actual UNCHANGED             7              10                  576

如有出手的指標
Accuracy: 0.7424242424242424

Precision: 0.8409090909090909

Recall: 0.3217391304347826


In [25]:
# 計算獲利
# 每次購買N張股票
def profit_calculator(df,N):
    profit_df = []
    cost_df = []
    revenue_df = []
    for i in range(0,len(df),1):
        if df.loc[i]['final_predict_sign'] == 'UP{}percent'.format(percentage_threshold*100):
            profit = (df.loc[i]['{}dayaf_high'.format(N_days_after)]- df.loc[i]['開盤價(元)'])*1000*N
            revenue = df.loc[i]['{}dayaf_high'.format(N_days_after)]*1000*N
            cost = df.loc[i]['開盤價(元)']*1000*N
        else:
            profit = 0
            revenue = 0
            cost = 0
            
        profit_df.append(profit)
        cost_df.append(cost)
        revenue_df.append(revenue)
    
    profit_df = np.array(profit_df)
    cost_df = np.array(cost_df)
    revenue_df = np.array(revenue_df)
        
    profit_sum = sum(profit_df)
    cost_sum = sum(cost_df)
    revenue_sum = sum(revenue_df)
    
    
    df_wrong_predict = (df.loc[(df['{}dayaf_sign_y'.format(N_days_after)] != 'UP{}percent'.format(percentage_threshold*100)) & (df['final_predict_sign'] == 'UP{}percent'.format(percentage_threshold*100))]['{}dayaf_high'.format(N_days_after)] - df.loc[(df['{}dayaf_sign_y'.format(N_days_after)] != 'UP{}percent'.format(percentage_threshold*100)) & (df['final_predict_sign'] == 'UP{}percent'.format(percentage_threshold*100))]['開盤價(元)']) * 1000 * N
    
    # Assuming cost_sum, profit_sum, and revenue_sum are your numeric values
    cost_sum_formatted = '{:,.0f}'.format(cost_sum)
    profit_sum_formatted = '{:,.0f}'.format(profit_sum)
    revenue_sum_formatted = '{:,.0f}'.format(revenue_sum)
    test_news_date = set(test_news['年月日'].reset_index(drop = True)[X_test_with_prob.index])
    
    report = {
        '預測股票 : ' : stock_want_to_predict,
        '資料集新聞篇數 : ' : len(df_wv_space_test) + len(df_wv_space_train),
        '資料集起始日期:': min(date_prediction_gb['年月日']),
        '資料集終止日期:' : max(date_prediction_gb['年月日']),
        '資料集有效日數:': len(date_prediction_gb),
        '\n測試新聞篇數 : ' : len(df_wv_space_test),
        '測試新聞起始日期:終止日期 :' : ((min(test_news_date))+ ' : ' + (max(test_news_date))), 
        '測試新聞有效日數:': len(test_news_date ),
        '-----------' : '-----------------------------',
        '總獲利(不考慮手續費與稅): $' : profit_sum_formatted,
        '總收入(不考慮手續費與稅): $' : revenue_sum_formatted,
        '總成本(不考慮手續費與稅): $' : cost_sum_formatted,
        'ROI : ' : '{:.2f}%'.format(round((revenue_sum - cost_sum)/cost_sum,4)*100) ,
        "\n平均單次成本 : $" : '{:,.0f}'.format(round(np.mean(cost_sum/len(cost_df[cost_df!=0])),0)), 
        '出手次數 : ' : len(cost_df[cost_df!=0]),
        '每次購買張數 : ' : N,
        '\n預測成功時平均賺 : $' :  '{:,.0f}'.format(round(np.mean(cost_df[cost_df>=0]),0)),
        '預測失敗時平均賺賠 : $' :  '{:,.0f}'.format(round(np.mean(df_wrong_predict),0)),
        '預測成功次數 : ' : confusion_df.iloc[0,0],
        '預測失敗次數 : ' : confusion_df.iloc[1,0] + confusion_df.iloc[2,0]      
    }
    
    for key, value in report.items():
        print(f"{key}{value}")
df = pd.merge(date_prediction_gb,specific_stock_price, left_on= '年月日',right_on= '年月日', how = 'left')

In [26]:
# Fianl Report
profit_calculator(df,2)

預測股票 : 台積電
資料集新聞篇數 : 16878
資料集起始日期:2016-01-04
資料集終止日期:2018-12-22
資料集有效日數:732

測試新聞篇數 : 3376
測試新聞起始日期:終止日期 :2016-03-08 : 2018-12-22
測試新聞有效日數:655
----------------------------------------
總獲利(不考慮手續費與稅): $719,500
總收入(不考慮手續費與稅): $19,134,520
總成本(不考慮手續費與稅): $18,415,020
ROI : 3.91%

平均單次成本 : $418,523
出手次數 : 44
每次購買張數 : 2

預測成功時平均賺 : $25,157
預測失敗時平均賺賠 : $4,791
預測成功次數 : 37
預測失敗次數 : 7
